In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import re

In [2]:
rejected_loan = pd.read_csv('C:/Users/ayan.pathak\Desktop/lending club loan data project/Data/rejected_2007_to_2018Q4.csv/rejected_2007_to_2018Q4.csv',low_memory=False)
print(f"Dataset shape: {rejected_loan.shape}")
print(f"Columns: {rejected_loan.columns.tolist()}")
print(f"Memory usage: {rejected_loan.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Dataset shape: (27648741, 9)
Columns: ['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score', 'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length', 'Policy Code']
Memory usage: 10795.32 MB


In [3]:
print(rejected_loan.head())
print(rejected_loan.dtypes.value_counts())

   Amount Requested Application Date                        Loan Title  \
0            1000.0       2007-05-26  Wedding Covered but No Honeymoon   
1            1000.0       2007-05-26                Consolidating Debt   
2           11000.0       2007-05-27       Want to consolidate my debt   
3            6000.0       2007-05-27                           waksman   
4            1500.0       2007-05-27                            mdrigo   

   Risk_Score Debt-To-Income Ratio Zip Code State Employment Length  \
0       693.0                  10%    481xx    NM           4 years   
1       703.0                  10%    010xx    MA          < 1 year   
2       715.0                  10%    212xx    MD            1 year   
3       698.0               38.64%    017xx    MA          < 1 year   
4       509.0                9.43%    209xx    MD          < 1 year   

   Policy Code  
0          0.0  
1          0.0  
2          0.0  
3          0.0  
4          0.0  
object     6
float64    3


In [4]:
# Check for missing values
missing_summary = rejected_loan.isnull().sum()
print(f"\nMissing values by column:")
for col, missing_count in missing_summary[missing_summary > 0].items():
    missing_pct = missing_count / len(rejected_loan) * 100
    print(f"  {col}: {missing_count:,} ({missing_pct:.1f}%)")


Missing values by column:
  Loan Title: 1,305 (0.0%)
  Risk_Score: 18,497,630 (66.9%)
  Zip Code: 293 (0.0%)
  State: 22 (0.0%)
  Employment Length: 951,355 (3.4%)
  Policy Code: 918 (0.0%)


In [5]:
# Check unique values for categorical columns
print("\n3. CATEGORICAL COLUMN ANALYSIS")
object_cols = rejected_loan.select_dtypes(include=['object']).columns
for col in object_cols:
    unique_count = rejected_loan[col].nunique()
    print(f"{col}: {unique_count} unique values")
    if unique_count < 10:
        print(f"  Values: {rejected_loan[col].unique().tolist()}")


3. CATEGORICAL COLUMN ANALYSIS
Application Date: 4238 unique values
Loan Title: 73927 unique values
Debt-To-Income Ratio: 126145 unique values
Zip Code: 1001 unique values
State: 51 unique values
Employment Length: 11 unique values


In [6]:
# A. Immediate column drops
print("A. Dropping unusable columns...")
cols_to_drop = []
if 'Risk_Score' in rejected_loan.columns:
    cols_to_drop.append('Risk_Score')
    print(f"  Dropping 'Risk_Score' - 66.9% missing, not in accepted data")
    
if 'Policy Code' in rejected_loan.columns:
    # Check if constant
    if rejected_loan['Policy Code'].nunique() == 1:
        cols_to_drop.append('Policy Code')
        print(f"  Dropping 'Policy Code' - constant value")

rejected_loan.drop(columns=cols_to_drop, inplace=True)
print(f"  Columns remaining: {rejected_loan.columns.tolist()}")

# B. Convert object columns to category where beneficial
print("\nB. Converting high-cardinality objects to category...")
object_cols = rejected_loan.select_dtypes(include=['object']).columns
for col in object_cols:
    unique_pct = rejected_loan[col].nunique() / len(rejected_loan) * 100
    if unique_pct < 50:  # Convert if < 50% unique values
        rejected_loan[col] = rejected_loan[col].astype('category')
        print(f"  {col}: object → category (saves ~50% memory)")

# C. Downcast numeric columns
print("\nC. Downcasting numeric columns...")
if 'Amount Requested' in rejected_loan.columns:
    # Most loan amounts are < 100k, use float32
    rejected_loan['Amount Requested'] = pd.to_numeric(
        rejected_loan['Amount Requested'], downcast='float'
    )
    print(f"  Amount Requested: {rejected_loan['Amount Requested'].dtype}")

print(f"\nMemory after optimization: {rejected_loan.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Shape: {rejected_loan.shape}")

A. Dropping unusable columns...
  Dropping 'Risk_Score' - 66.9% missing, not in accepted data
  Columns remaining: ['Amount Requested', 'Application Date', 'Loan Title', 'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length', 'Policy Code']

B. Converting high-cardinality objects to category...
  Application Date: object → category (saves ~50% memory)
  Loan Title: object → category (saves ~50% memory)
  Debt-To-Income Ratio: object → category (saves ~50% memory)
  Zip Code: object → category (saves ~50% memory)
  State: object → category (saves ~50% memory)
  Employment Length: object → category (saves ~50% memory)

C. Downcasting numeric columns...
  Amount Requested: float64

Memory after optimization: 810.75 MB
Shape: (27648741, 8)


In [7]:
# First, parse dates to understand temporal distribution
if 'Application Date' in rejected_loan.columns:
    print("Analyzing temporal distribution...")
    rejected_loan['app_date_parsed'] = pd.to_datetime(
        rejected_loan['Application Date'], errors='coerce'
    )
    
    # Extract year for stratified sampling
    rejected_loan['app_year'] = rejected_loan['app_date_parsed'].dt.year
    
    print("Yearly distribution:")
    year_counts = rejected_loan['app_year'].value_counts().sort_index()
    for year, count in year_counts.head(10).items():
        if not pd.isna(year):
            print(f"  {year}: {count:,} applications ({count/len(rejected_loan)*100:.1f}%)")

Analyzing temporal distribution...
Yearly distribution:
  2007: 5,274 applications (0.0%)
  2008: 25,596 applications (0.1%)
  2009: 56,991 applications (0.2%)
  2010: 112,561 applications (0.4%)
  2011: 217,792 applications (0.8%)
  2012: 337,277 applications (1.2%)
  2013: 760,942 applications (2.8%)
  2014: 1,933,700 applications (7.0%)
  2015: 2,859,379 applications (10.3%)
  2016: 4,769,874 applications (17.3%)


In [8]:
# STRATEGIC SAMPLING APPROACH
print("\nChoosing sampling strategy...")
print("Option 1: Simple random sample (fast, loses temporal patterns)")
print("Option 2: Stratified by year (preserves time distribution)")
print("Option 3: Focused on overlapping years with accepted loans")

# Check date overlap with accepted loans
print("\nChecking temporal overlap with accepted loans...")
# Accepted loans: 2007-06-01 to 2018-12-01 (from your output)
accepted_start = pd.Timestamp('2007-06-01')
accepted_end = pd.Timestamp('2018-12-01')


Choosing sampling strategy...
Option 1: Simple random sample (fast, loses temporal patterns)
Option 2: Stratified by year (preserves time distribution)
Option 3: Focused on overlapping years with accepted loans

Checking temporal overlap with accepted loans...


In [9]:
# parse dates properly
if 'Application Date' in rejected_loan.columns:
    print("Parsing application dates...")
    # Parse to datetime WITHOUT converting to categorical
    rejected_loan['app_date_parsed'] = pd.to_datetime(
        rejected_loan['Application Date'], errors='coerce'
    )
    
    # Convert to datetime type explicitly
    rejected_loan['app_date_parsed'] = rejected_loan['app_date_parsed'].astype('datetime64[ns]')
    
    # Get date range (handle NaN values)
    valid_dates = rejected_loan['app_date_parsed'].dropna()
    if not valid_dates.empty:
        rejected_start = valid_dates.min()
        rejected_end = valid_dates.max()
        print(f"Rejected date range: {rejected_start} to {rejected_end}")
        
        # Check overlap with accepted loans timeframe
        accepted_start = pd.Timestamp('2007-06-01')
        accepted_end = pd.Timestamp('2018-12-01')
        print(f"Accepted date range: {accepted_start} to {accepted_end}")
        
        # Calculate overlap
        overlap_mask = rejected_loan['app_date_parsed'].between(accepted_start, accepted_end)
        overlap_count = overlap_mask.sum()
        overlap_pct = overlap_count / len(rejected_loan) * 100
        print(f"Applications in accepted time range: {overlap_count:,} ({overlap_pct:.1f}%)")
    else:
        print("No valid dates found in Application Date column")
        overlap_mask = pd.Series(False, index=rejected_loan.index)
        overlap_count = 0

Parsing application dates...
Rejected date range: 2007-05-26 00:00:00 to 2018-12-31 00:00:00
Accepted date range: 2007-06-01 00:00:00 to 2018-12-01 00:00:00
Applications in accepted time range: 26,834,620 (97.1%)


In [10]:
target_sample = 2260000  # Match accepted loans
current_size = len(rejected_loan)
sampling_ratio = target_sample / current_size

print(f"\nCurrent size: {current_size:,}")
print(f"Target sample: {target_sample:,}")
print(f"Sampling ratio: {sampling_ratio:.3f}")


Current size: 27,648,741
Target sample: 2,260,000
Sampling ratio: 0.082


In [11]:
if overlap_count >= target_sample * 0.8:  # If sufficient overlap (80%+ of target)
    print("Sufficient data in overlapping period")
    overlap_data = rejected_loan[overlap_mask].copy()
    
    if len(overlap_data) > target_sample:
        # Add year column for stratified sampling
        overlap_data['app_year'] = overlap_data['app_date_parsed'].dt.year
        
        # Remove rows with NaN years
        overlap_data = overlap_data.dropna(subset=['app_year'])
        
        if overlap_data['app_year'].nunique() > 1:
            print("Stratified sampling by year within overlap period...")
            # Calculate per-year sampling ratio
            sampled_data = overlap_data.groupby('app_year', group_keys=False).apply(
                lambda x: x.sample(frac=sampling_ratio, random_state=42, replace=False)
            )
        else:
            # Simple random sample
            sampled_data = overlap_data.sample(n=target_sample, random_state=42)
        
        print(f"Sampled {len(sampled_data):,} from overlap period")
        rejected_loan = sampled_data
    else:
        print(f"Using all overlap data: {len(overlap_data):,} rows")
        rejected_loan = overlap_data

else:
    print("Insufficient overlap, sampling from full dataset")
    
    if 'app_date_parsed' in rejected_loan.columns and rejected_loan['app_date_parsed'].notna().any():
        # Add year column
        rejected_loan['app_year'] = rejected_loan['app_date_parsed'].dt.year
        valid_data = rejected_loan.dropna(subset=['app_year'])
        
        if valid_data['app_year'].nunique() > 1:
            print("Stratified sampling by year...")
            sampled = valid_data.groupby('app_year', group_keys=False).apply(
                lambda x: x.sample(frac=sampling_ratio, random_state=42, replace=False)
            )
            rejected_loan = sampled
        else:
            # Simple random sample
            rejected_loan = rejected_loan.sample(n=target_sample, random_state=42)
    else:
        # Simple random sample
        rejected_loan = rejected_loan.sample(n=target_sample, random_state=42)

print(f"\nShape after sampling: {rejected_loan.shape}")

Sufficient data in overlapping period
Stratified sampling by year within overlap period...
Sampled 2,193,453 from overlap period

Shape after sampling: (2193453, 10)


In [12]:
# A. Drop temporary columns
cols_to_drop = []
if 'app_year' in rejected_loan.columns:
    cols_to_drop.append('app_year')
if 'app_date_parsed' in rejected_loan.columns:
    # We'll keep the parsed date but rename it
    pass  # We'll handle this in renaming

if cols_to_drop:
    rejected_loan.drop(columns=cols_to_drop, inplace=True)
    print(f"Dropped temporary columns: {cols_to_drop}")

Dropped temporary columns: ['app_year']


In [13]:
# Rename columns to match accepted loans naming convention
print("\nStandardizing column names...")
column_rename_map = {
    'Amount Requested': 'loan_amnt',
    'Application Date': 'application_date',  # Will convert to issue_d equivalent
    'Loan Title': 'title',
    'Debt-To-Income Ratio': 'dti',
    'Zip Code': 'zip_code',
    'State': 'addr_state',
    'Employment Length': 'emp_length'
}

# Only rename columns that exist
existing_renames = {k: v for k, v in column_rename_map.items() if k in rejected_loan.columns}
rejected_loan.rename(columns=existing_renames, inplace=True)
print(f"Renamed columns: {existing_renames}")


Standardizing column names...
Renamed columns: {'Amount Requested': 'loan_amnt', 'Application Date': 'application_date', 'Loan Title': 'title', 'Debt-To-Income Ratio': 'dti', 'Zip Code': 'zip_code', 'State': 'addr_state', 'Employment Length': 'emp_length'}


In [14]:
# Convert 'application_date' to match accepted's 'issue_d_dt' format
if 'application_date' in rejected_loan.columns:
    print("\nC. Standardizing date column...")
    # Convert to datetime (ensure consistency)
    rejected_loan['application_date'] = pd.to_datetime(
        rejected_loan['application_date'], errors='coerce'
    )
    # Rename to match accepted format
    rejected_loan.rename(columns={'application_date': 'issue_d_dt'}, inplace=True)
    print(f"Date column standardized as 'issue_d_dt'")
    print(f"Date range: {rejected_loan['issue_d_dt'].min()} to {rejected_loan['issue_d_dt'].max()}")
    print(f"Missing dates: {rejected_loan['issue_d_dt'].isnull().sum()}")


C. Standardizing date column...
Date column standardized as 'issue_d_dt'
Date range: 2007-06-01 00:00:00 to 2018-12-01 00:00:00
Missing dates: 0


In [15]:
# Clean 'dti' column (convert percentages to numeric)
if 'dti' in rejected_loan.columns:
    print("\nCleaning DTI column...")
    
    # First, convert categorical back to string for processing
    if rejected_loan['dti'].dtype.name == 'category':
        print("  Converting categorical DTI to string...")
        rejected_loan['dti'] = rejected_loan['dti'].astype(str)
    
    # Check current format
    dti_sample = rejected_loan['dti'].head(3).tolist()
    print(f"  DTI sample values: {dti_sample}")
    
    # Function to clean DTI values
    def clean_dti_value(val):
        if pd.isna(val) or val == 'nan' or val == 'None' or val == 'null':
            return np.nan
        if isinstance(val, str):
            # Remove % sign and spaces
            val = val.replace('%', '').strip()
            if val == '' or val.lower() == 'nan':
                return np.nan
            # Handle edge cases
            if val.startswith('<'):
                val = val[1:]
            if val.startswith('>'):
                val = val[1:]
        try:
            float_val = float(val)
            return float_val
        except (ValueError, TypeError):
            return np.nan
    
    # Apply cleaning
    rejected_loan['dti'] = rejected_loan['dti'].apply(clean_dti_value)
    
    dti_notna = rejected_loan['dti'].dropna()
    if not dti_notna.empty:
        print(f"  DTI statistics after cleaning:")
        print(f"    Range: {dti_notna.min():.2f} to {dti_notna.max():.2f}")
        print(f"    Mean: {dti_notna.mean():.2f}, Median: {dti_notna.median():.2f}")
        print(f"    Std: {dti_notna.std():.2f}")
    else:
        print("  Warning: No valid DTI values after cleaning")
    
    missing_count = rejected_loan['dti'].isnull().sum()
    missing_pct = missing_count / len(rejected_loan) * 100
    print(f"  Missing: {missing_count:,} ({missing_pct:.2f}%)")


Cleaning DTI column...
  Converting categorical DTI to string...
  DTI sample values: ['0.6%', '66.15%', '0%']
  DTI statistics after cleaning:
    Range: -1.00 to 2376000.00
    Mean: 143.90, Median: 19.94
    Std: 4032.21
  Missing: 0 (0.00%)


In [16]:
# Standardize 'emp_length' to match accepted format
if 'emp_length' in rejected_loan.columns:
    print("\nStandardizing employment length...")
    
    # Convert to string if categorical
    if rejected_loan['emp_length'].dtype.name == 'category':
        rejected_loan['emp_length'] = rejected_loan['emp_length'].astype(str)
    
    # Check current format
    unique_emp = rejected_loan['emp_length'].dropna().unique()[:5]
    print(f"  Sample employment length values: {unique_emp}")
    
    # Standardization mapping
    emp_standardization = {
        '< 1 year': '< 1 year',
        '1 year': '1 year',
        '2 years': '2 years',
        '3 years': '3 years',
        '4 years': '4 years',
        '5 years': '5 years',
        '6 years': '6 years',
        '7 years': '7 years',
        '8 years': '8 years',
        '9 years': '9 years',
        '10+ years': '10+ years',
        '10 years': '10+ years',
        '10+years': '10+ years',
        '<1 year': '< 1 year',
        '1 Year': '1 year',
        '2 Years': '2 years',
        '3 Years': '3 years',
        '4 Years': '4 years',
        '5 Years': '5 years',
        '6 Years': '6 years',
        '7 Years': '7 years',
        '8 Years': '8 years',
        '9 Years': '9 years',
        '10 Years': '10+ years',
        'n/a': np.nan,
        'nan': np.nan,
        'None': np.nan,
        '': np.nan
    }
    
    # Apply standardization
    rejected_loan['emp_length'] = rejected_loan['emp_length'].map(emp_standardization)
    
    print("  Employment length distribution after standardization:")
    emp_dist = rejected_loan['emp_length'].value_counts(dropna=False).head(10)
    for val, count in emp_dist.items():
        pct = count / len(rejected_loan) * 100
        val_display = 'NaN' if pd.isna(val) else val
        print(f"    {val_display}: {count:,} ({pct:.1f}%)")


Standardizing employment length...
  Sample employment length values: ['< 1 year' '10+ years' '1 year' '3 years' '5 years']
  Employment length distribution after standardization:
    < 1 year: 1,823,331 (83.1%)
    5 years: 184,003 (8.4%)
    NaN: 74,911 (3.4%)
    10+ years: 33,514 (1.5%)
    1 year: 19,956 (0.9%)
    2 years: 15,456 (0.7%)
    3 years: 13,601 (0.6%)
    4 years: 9,393 (0.4%)
    6 years: 5,700 (0.3%)
    8 years: 5,264 (0.2%)


In [17]:
# Handle missing values
print("\nHandling missing values...")
missing_before = rejected_loan.isnull().sum().sum()
print(f"  Total missing values before imputation: {missing_before:,}")


Handling missing values...
  Total missing values before imputation: 75,100


In [18]:
# For columns with minimal missing, use appropriate imputation
for col in rejected_loan.columns:
    if rejected_loan[col].isnull().any():
        missing_pct = rejected_loan[col].isnull().mean()
        
        if missing_pct < 0.01:  # Less than 1% missing
            if rejected_loan[col].dtype in ['float32', 'float64', 'int32', 'int64']:
                # Numeric column: impute with median
                median_val = rejected_loan[col].median()
                rejected_loan[col] = rejected_loan[col].fillna(median_val)
                print(f"    {col}: Imputed {missing_pct*100:.2f}% missing with median {median_val}")
            elif rejected_loan[col].dtype == 'object' or rejected_loan[col].dtype.name == 'category':
                # Categorical: impute with mode
                mode_val = rejected_loan[col].mode()[0] if not rejected_loan[col].mode().empty else 'Unknown'
                rejected_loan[col] = rejected_loan[col].fillna(mode_val)
                print(f"    {col}: Imputed {missing_pct*100:.2f}% missing with mode '{mode_val}'")

    title: Imputed 0.00% missing with mode 'Debt consolidation'
    zip_code: Imputed 0.00% missing with mode '112xx'
    Policy Code: Imputed 0.00% missing with median 0.0


In [19]:
# Final missing check
missing_after = rejected_loan.isnull().sum().sum()
print(f"  Total missing values after imputation: {missing_after:,}")

# Final dataset info
print("\n FINAL DATASET SUMMARY")
print(f"  Shape: {rejected_loan.shape}")
print(f"  Columns: {rejected_loan.columns.tolist()}")
print(f"  Data types:")
print(rejected_loan.dtypes.value_counts())
print(f"  Memory usage: {rejected_loan.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

  Total missing values after imputation: 74,911

 FINAL DATASET SUMMARY
  Shape: (2193453, 9)
  Columns: ['loan_amnt', 'issue_d_dt', 'title', 'dti', 'zip_code', 'addr_state', 'emp_length', 'Policy Code', 'app_date_parsed']
  Data types:
float64           3
datetime64[ns]    2
category          1
category          1
category          1
object            1
Name: count, dtype: int64
  Memory usage: 256.00 MB


In [20]:
rejected_loan.head()

,loan_amnt,issue_d_dt,title,dti,zip_code,addr_state,emp_length,Policy Code,app_date_parsed
471,3500.0,2007-07-14,daniel09,0.60,322xx,FL,< 1 year,0.0,2007-07-14
3280,5000.0,2007-11-27,home_improvement,66.15,337xx,FL,10+ years,0.0,2007-11-27
72,2500.0,2007-06-02,moecheeks21,0.00,674xx,KS,< 1 year,0.0,2007-06-02
4083,1500.0,2007-12-09,hotsweetmami143,1.50,114xx,NY,< 1 year,0.0,2007-12-09
2006,15000.0,2007-10-24,Jay3122,5.21,334xx,FL,1 year,0.0,2007-10-24


In [22]:
print("\n7. CRITICAL DTI DATA QUALITY INVESTIGATION")

# Investigate DTI distribution
if 'dti' in rejected_loan.columns:
    print("\nA. DTI Distribution Analysis:")
    
    # Check value counts for common ranges
    print("DTI value ranges:")
    bins = [0, 10, 20, 30, 40, 50, 100, 500, 1000, 5000, 10000, 50000, 100000, 1000000, float('inf')]
    labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-100', '100-500', '500-1k', 
              '1k-5k', '5k-10k', '10k-50k', '50k-100k', '100k-1M', '>1M']
    
    dti_ranges = pd.cut(rejected_loan['dti'], bins=bins, labels=labels, include_lowest=True)
    range_counts = dti_ranges.value_counts().sort_index()
    
    for rng, count in range_counts.items():
        if count > 0:
            pct = count / len(rejected_loan) * 100
            print(f"  {rng}: {count:,} ({pct:.2f}%)")
    
    # Investigate extreme values
    print("\nB. Investigating extreme DTI values:")
    
    # Check if DTI might be mislabeled (maybe it's income or loan amount?)
    # Compare with loan_amnt to see correlation
    if 'loan_amnt' in rejected_loan.columns:
        # Sample extreme DTI rows
        extreme_dti = rejected_loan[rejected_loan['dti'] > 100].head(5)
        print("Sample rows with DTI > 100:")
        for idx, row in extreme_dti.iterrows():
            print(f"  Loan: ${row['loan_amnt']:,.0f}, DTI: {row['dti']:,.0f}, Title: {row.get('title', 'N/A')}")
    
    # Check for data type confusion
    print("\nC. Checking for data type issues:")
    print(f"DTI dtype: {rejected_loan['dti'].dtype}")
    
    # Look at the raw values before cleaning
    print("\nExamining raw DTI string patterns (before conversion):")
    # We need to reload or examine original data pattern


7. CRITICAL DTI DATA QUALITY INVESTIGATION

A. DTI Distribution Analysis:
DTI value ranges:
  0-10: 535,460 (24.41%)
  10-20: 466,027 (21.25%)
  20-30: 370,177 (16.88%)
  30-40: 251,414 (11.46%)
  40-50: 155,425 (7.09%)
  50-100: 252,089 (11.49%)
  100-500: 44,028 (2.01%)
  500-1k: 7,007 (0.32%)
  1k-5k: 3,245 (0.15%)
  5k-10k: 6,864 (0.31%)
  10k-50k: 2,560 (0.12%)
  50k-100k: 791 (0.04%)
  100k-1M: 171 (0.01%)
  >1M: 3 (0.00%)

B. Investigating extreme DTI values:
Sample rows with DTI > 100:
  Loan: $10,000, DTI: 146, Title: MTM21o83
  Loan: $5,000, DTI: 271, Title: michele
  Loan: $5,000, DTI: 2,527, Title: CRodney
  Loan: $7,500, DTI: 175, Title: CinLou114
  Loan: $2,000, DTI: 239, Title: thejuug

C. Checking for data type issues:
DTI dtype: float64

Examining raw DTI string patterns (before conversion):


In [23]:
# Cell 8: Intelligent DTI Processing
print("\n8. INTELLIGENT DTI PROCESSING")

# Create DTI categories based on distribution
rejected_loan['dti_category'] = pd.cut(
    rejected_loan['dti'],
    bins=[-1, 0, 10, 20, 30, 40, 50, 100, float('inf')],
    labels=['0', '0-10', '10-20', '20-30', '30-40', '40-50', '50-100', '>100'],
    include_lowest=True
)

print("DTI Category Distribution:")
category_counts = rejected_loan['dti_category'].value_counts().sort_index()
for cat, count in category_counts.items():
    pct = count / len(rejected_loan) * 100
    print(f"  {cat}: {count:,} ({pct:.2f}%)")

# Create flag for extreme DTI values
rejected_loan['dti_extreme'] = (rejected_loan['dti'] > 100).astype(int)
print(f"\nExtreme DTI (>100) flag: {rejected_loan['dti_extreme'].sum():,} rows ({rejected_loan['dti_extreme'].mean()*100:.2f}%)")

# Cap DTI at 100 for modeling, but keep original for reference
rejected_loan['dti_capped'] = rejected_loan['dti'].clip(upper=100, lower=0)

print("\nDTI Statistics (capped at 100):")
capped_stats = rejected_loan['dti_capped'].describe()
print(f"  Min: {capped_stats['min']:.2f}")
print(f"  25%: {capped_stats['25%']:.2f}")
print(f"  Median: {capped_stats['50%']:.2f}")
print(f"  75%: {capped_stats['75%']:.2f}")
print(f"  Max: {capped_stats['max']:.2f}")
print(f"  Mean: {capped_stats['mean']:.2f}")

# Keep original DTI for anomaly investigation
rejected_loan['dti_original'] = rejected_loan['dti']
rejected_loan['dti'] = rejected_loan['dti_capped']  # Use capped for modeling


8. INTELLIGENT DTI PROCESSING
DTI Category Distribution:
  0: 180,853 (8.25%)
  0-10: 452,799 (20.64%)
  10-20: 466,027 (21.25%)
  20-30: 370,177 (16.88%)
  30-40: 251,414 (11.46%)
  40-50: 155,425 (7.09%)
  50-100: 252,089 (11.49%)
  >100: 64,669 (2.95%)

Extreme DTI (>100) flag: 64,669 rows (2.95%)

DTI Statistics (capped at 100):
  Min: 0.00
  25%: 8.02
  Median: 19.94
  75%: 36.50
  Max: 100.00
  Mean: 27.43


In [24]:
print("\n EMPLOYMENT LENGTH CORRECTION & FINAL CLEANUP")

# A. Compare with accepted loans distribution
print("A. Validating employment length distribution...")

# Load accepted loans for comparison
try:
    accepted_data = pd.read_parquet('accepted_loan_preprocessed.parquet', engine='fastparquet', columns=['emp_length'])
    print("\nAccepted loans employment length distribution:")
    accepted_dist = accepted_data['emp_length'].value_counts(normalize=True).sort_index() * 100
    for val, pct in accepted_dist.items():
        print(f"  {val}: {pct:.1f}%")
except Exception as e:
    print(f"Could not load accepted data: {e}")
    print("Using domain knowledge: '10+ years' should be largest category")

# B. Create corrected employment categories
print("\nB. Creating employment length categories...")

# Current distribution seems off (83% < 1 year is unrealistic)
# Create broader categories
employment_mapping = {
    '< 1 year': '0-1 years',
    '1 year': '1-3 years',
    '2 years': '1-3 years',
    '3 years': '1-3 years',
    '4 years': '4-6 years',
    '5 years': '4-6 years',
    '6 years': '4-6 years',
    '7 years': '7-9 years',
    '8 years': '7-9 years',
    '9 years': '7-9 years',
    '10+ years': '10+ years',
    np.nan: 'Unknown'
}

rejected_loan['emp_length_category'] = rejected_loan['emp_length'].map(employment_mapping)

print("Employment length categories:")
cat_dist = rejected_loan['emp_length_category'].value_counts(normalize=True).sort_index() * 100
for cat, pct in cat_dist.items():
    print(f"  {cat}: {pct:.1f}%")

# C. Final column cleanup
print("\nC. Final column selection...")

# Select columns for anomaly detection
final_columns = [
    'loan_amnt',
    'issue_d_dt',
    'dti',  # Capped version (0-100)
    'dti_category',
    'dti_extreme',  # Flag for >100
    'addr_state',
    'emp_length_category',
    'title'  # Keep for potential text analysis
]

# Add any other relevant columns
available_final = [col for col in final_columns if col in rejected_loan.columns]
print(f"Selected {len(available_final)} columns for anomaly detection:")
print(f"  {available_final}")

# Create final dataset
rejected_final = rejected_loan[available_final].copy()

# D. Save final datasets
print("\nD. Saving final datasets...")

# Save full cleaned dataset
rejected_loan.to_parquet('rejected_loan_full_cleaned.parquet', engine='fastparquet')
print("Saved full cleaned dataset: 'rejected_loan_full_cleaned.parquet'")

# Save modeling-ready dataset
rejected_final.to_parquet('rejected_loan_modeling_ready.parquet', engine='fastparquet')
print("Saved modeling-ready dataset: 'rejected_loan_modeling_ready.parquet'")

# E. Final summary
print("\n=== FINAL SUMMARY ===")
print(f"1. Total rows: {len(rejected_final):,}")
print(f"2. Features: {rejected_final.shape[1]}")
print(f"3. Memory: {rejected_final.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"4. Time range: {rejected_final['issue_d_dt'].min()} to {rejected_final['issue_d_dt'].max()}")

# Show data quality metrics
print("\n5. Data Quality Metrics:")
if 'dti_extreme' in rejected_final.columns:
    extreme_pct = rejected_final['dti_extreme'].mean() * 100
    print(f"   DTI >100 values: {extreme_pct:.2f}% (anomaly candidates)")

if 'emp_length_category' in rejected_final.columns:
    unknown_emp = (rejected_final['emp_length_category'] == 'Unknown').mean() * 100
    print(f"   Unknown employment: {unknown_emp:.2f}%")

print("\n REJECTED DATA READY FOR ANOMALY DETECTION!")


 EMPLOYMENT LENGTH CORRECTION & FINAL CLEANUP
A. Validating employment length distribution...

Accepted loans employment length distribution:
  1 year: 6.6%
  10+ years: 33.1%
  2 years: 9.0%
  3 years: 8.0%
  4 years: 6.0%
  5 years: 6.2%
  6 years: 4.5%
  7 years: 4.1%
  8 years: 4.1%
  9 years: 3.5%
  < 1 year: 8.4%
  Not Provided: 6.5%

B. Creating employment length categories...
Employment length categories:
  0-1 years: 83.1%
  1-3 years: 2.2%
  10+ years: 1.5%
  4-6 years: 9.1%
  7-9 years: 0.6%
  Unknown: 3.4%

C. Final column selection...
Selected 8 columns for anomaly detection:
  ['loan_amnt', 'issue_d_dt', 'dti', 'dti_category', 'dti_extreme', 'addr_state', 'emp_length_category', 'title']

D. Saving final datasets...
Saved full cleaned dataset: 'rejected_loan_full_cleaned.parquet'
Saved modeling-ready dataset: 'rejected_loan_modeling_ready.parquet'

=== FINAL SUMMARY ===
1. Total rows: 2,193,453
2. Features: 8
3. Memory: 241.63 MB
4. Time range: 2007-06-01 00:00:00 to 2018

In [54]:
# Cell 1: Test config
from config import check_paths
check_paths()

# Cell 2: Load data
from config import load_dataset
accepted_data = load_dataset('accepted_preprocessed')

# Cell 3: Quick check
print(f"Shape: {accepted_data.shape}")
if 'loan_status' in accepted_data.columns:
    print("Loan status distribution:")
    print(accepted_data['loan_status'].value_counts(normalize=True).head())
    

=== PROJECT PATH VERIFICATION ===
  ✅ Notebook Folder: Directory
  ✅ Project Folder: Directory
  ✅ Raw Data Directory: Directory
  ✅ Raw Accepted Data: 1597.5 MB
  ✅ Raw Rejected Data: 1699.7 MB
  ✅ Processed Data Directory: Directory
  ✅ Processed Accepted: 293.3 MB
  ✅ Processed Rejected: 40.4 MB

🎉 ALL CRITICAL PATHS VERIFIED!
Ready for anomaly detection.
📂 Loading accepted_preprocessed...
   Path: c:\\Users\\ayan.pathak\\Desktop\\lending club loan data project\\Notebook\processed_data\accepted\accepted_loan_preprocessed.parquet
✅ Loaded: 2,260,639 rows × 88 columns
   Memory: 715.77 MB
Shape: (2260639, 88)
Loan status distribution:
loan_status
Fully Paid            0.476304
Current               0.388526
Charged Off           0.118798
Late (31-120 days)    0.009496
In Grace Period       0.003732
Name: proportion, dtype: float64
